In [1]:
import pandas as pd
import folium
import json
import os
import sys
sys.path.append('../..')
from src.db.db_handler import DBHandler
from helpers.maps import get_geojson

In [2]:
db = DBHandler()

In [3]:
estaciones = db.get_columns("estaciones", ['idema', 'provincia_id'])
provincias = db.get_columns("provincias", ['id', 'nombre', 'com_auto_id'])
comunidades = db.get_columns("comunidades_autonomas", ['id', 'nombre'])

In [5]:
estaciones_df = pd.DataFrame(estaciones)
provincias_df = pd.DataFrame(provincias)
comunidades_df = pd.DataFrame(comunidades)

In [4]:
temperatura = db.get_columns("temperatura_historico", ['fecha', 'idema','tmed', 'tmin', 'tmax'])
lluvia = db.get_columns("lluvia_historico", ['fecha', 'idema','prec'])
viento = db.get_columns("viento_historico", ['fecha', 'idema','velmedia', 'racha'])
humedad = db.get_columns("humedad_historico", ['fecha', 'idema','hr_media', 'hr_min', 'hr_max'])

In [6]:
provincias_df.rename(columns={'id': 'provincia_id', 'nombre': 'provincia'}, inplace=True)
comunidades_df.rename(columns={'id': 'com_auto_id', 'nombre': 'comunidad'}, inplace=True)

In [9]:
geodata_provincias = get_geojson()
geodata_provincias = [v['geojson'] for k, v in geodata_provincias.items()]

In [12]:
features_provincias = []
for data in geodata_provincias:
    if data['features']:
        features_provincias.extend(data['features'])

In [13]:
geojson_provincias = {
    "type": "FeatureCollection",
    "features": features_provincias
}

In [30]:
def create_avg_df(data):
    df = pd.DataFrame(data)
    df = df.merge(estaciones_df, on='idema', how='left')
    df = df.merge(provincias_df, on='provincia_id', how='left')
    df = df.merge(comunidades_df, on='com_auto_id', how='left')
    df['provincia_id'] = df['provincia_id'].astype(str)
    for col in df.columns:
        if col not in ['fecha', 'idema']:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df.drop(columns=['fecha', 'idema']).groupby('provincia_id').mean().reset_index()

In [19]:
def add_to_map(m, avg_df, col, label, color):
    folium.Choropleth(
        geo_data=geojson_provincias,
        data=avg_df,
        columns=['provincia_id', col],
        key_on="properties.provincia_id",
        fill_color=color,
        fill_opacity=1.0,
        line_opacity=0.2,
        legend_name=label,
        highlight=True,
        reset_style_on_click=True
    ).add_to(m)

In [38]:
temperatura_avg = create_avg_df(temperatura)
m = folium.Map(location=[40, -2], zoom_start=6)
add_to_map(m, temperatura_avg, 'tmed', "Average Temperature (°C)", 'Reds')
add_to_map(m, temperatura_avg, 'tmin', "Minimum Temperature (°C)", 'Blues')
add_to_map(m, temperatura_avg, 'tmax', "Maximum Temperature (°C)", 'YlOrRd')
folium.LayerControl().add_to(m)
m

In [39]:
lluvia_avg = create_avg_df(lluvia)
m = folium.Map(location=[40, -2], zoom_start=6)
add_to_map(m, lluvia_avg, 'prec', "Precipitación", 'Blues')
folium.LayerControl().add_to(m)
m

In [41]:
m = folium.Map(location=[40, -2], zoom_start=6)
humedad_avg = create_avg_df(humedad)
add_to_map(m, humedad_avg, 'hr_media', "Humedad Relativa", 'BuGn')
folium.LayerControl().add_to(m)
m

In [42]:
m = folium.Map(location=[40, -2], zoom_start=6)
viento_avg = create_avg_df(viento)
add_to_map(m, viento_avg, 'velmedia', "Velocidad Media del Viento", 'YlGn')
add_to_map(m, viento_avg, 'racha', "Racha Viento", 'Greens')
folium.LayerControl().add_to(m)
m